In [1]:
from groq import Groq

In [ ]:
# Functions
def load_note(): # TODO: link to the doctor's actual input
    # return a string with the doctor note
    return "This patient's history of recurrent kidney stones, osteopenia, and high-dose vitamin D supplementation, along with laboratory findings of hypercalcemia and hypercalciuria, suggest the possibility of vitamin D toxicity. Excessive intake of vitamin D can cause increased absorption of calcium from the gut, leading to hypercalcemia and hypercalciuria, which can result in kidney stones and bone loss. Treatment would involve stopping the vitamin D supplementation and potentially providing intravenous fluids and loop diuretics to promote the excretion of calcium."

def extract_pathologies(completions, batch_size=1):
    results = []
    total = len(completions)
    for i in range(0, total, batch_size):
        batch = completions[i:i+batch_size]        
        prompt = "Given the following medical case descriptions, extract and return only the main pathology or condition mentioned for each case (one per line, one per case, in order). If the pathology is very unclear, return 'unknown', if the text is unrelated to a pathology description, then return 'unrelated'. DON'T RETURN ANYTHING ELSE BUT THE PATHOLOGIES:\n\n"
        for j, completion in enumerate(batch, 1):
            prompt += f"CASE {j}:\n{completion}\n\n"
        prompt += """
        EXAMPLE INPUT:
        CASE 1: This patient's history of recurrent kidney stones, osteopenia, and high-dose vitamin D supplementation, along with laboratory findings of hypercalcemia and hypercalciuria, suggest the possibility of vitamin D toxicity. Excessive intake of vitamin D can cause increased absorption of calcium from the gut, leading to hypercalcemia and hypercalciuria, which can result in kidney stones and bone loss. Treatment would involve stopping the vitamin D supplementation and potentially providing intravenous fluids and loop diuretics to promote the excretion of calcium.
        CASE 2: A 45-year-old patient presents with vague symptoms including fatigue, mild joint pain, and occasional headaches. Physical examination reveals no significant findings, and initial laboratory tests are within normal ranges. The patient's medical history is unremarkable, and they report no recent travel or exposure to infectious diseases. Despite thorough questioning, no clear pattern or specific triggers for the symptoms can be identified.
        CASE 2: This child's symptoms of a red, bulging tympanic membrane with systemic symptoms such as fever and headache, and the additional symptoms of dizziness and a spinning sensation, raise concern for complications of acute otitis media. The differential diagnosis could include labyrinthitis or possibly even mastoiditis. Urgent evaluation, including further imaging studies such as a CT or MRI scan, may be necessary. This child likely requires admission for intravenous antibiotics and possibly surgical intervention if mastoiditis is confirmed.
        CASE 4: While the symptoms might initially suggest asthma, the lack of response to bronchodilators indicates a different cause. A possible diagnosis in this case might be idiopathic pulmonary fibrosis, a type of lung disease that results in scarring (fibrosis) of the lungs for an unknown reason. High-resolution CT of the chest would be the next step in diagnosis.
        CASE 5: The annual report for XYZ Corporation shows a 15% increase in revenue compared to the previous fiscal year. The company attributes this growth to the successful launch of a new product line and expansion into international markets. The board of directors has approved a 5% dividend increase for shareholders. The report also outlines plans for sustainable practices and corporate social responsibility initiatives for the coming year.

        EXAMPLE OUTPUT:
        vitamin d toxicity
        unknown
        acute otitis media, possible complication of labyrinthitis or mastoiditis
        idiopathic pulmonary fibrosis
        unrelated
        """
        

        client = Groq(api_key="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")
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama-3.1-70b-versatile",
        )
        
        batch_results = response.choices[0].message.content.strip().split('\n')
        results.extend(batch_results)
    
    return results


In [ ]:
class Patient:
    def __init__(self, id, active_pathologies=[]):
        self.id = id
        self.active_pathologies = active_pathologies


    def add_pathologies(self, pathologies):
        """Adds new pathologies to the patient's active pathologies and returns the new complete set

        Args:
            pathologies (_type_): _description_

        Returns:
            _type_: _description_
        """
        self.active_pathologies.extend(pathologies)
        return self.active_pathologies
        
    def remove_pathology(self, pathology):
        self.active_pathologies.remove(pathology)

In [ ]:
patient = Patient(1)

# Main flow
doctor_note = load_note()

# Extract pathology from doctor note
pathologies = extract_pathologies(doctor_note)

# If new pathologies are found, add it to the active pathologies of the patient
if new_pathologies := patient.has_new_pathologies(pathologies):
    # Add new pathology to the patient's active pathologies
    pathologies = patient.active_pathologies.add_pathologies(new_pathologies)
    
# Extract features that are trackable, and relate them to the active pathologies
# features = extract_features(doctor_note)

# TODO: Modify pathologies + features to be saved (human in the loop)

# Save features and pathologies to DB
# save_treatment_data(patient, features, pathologies)